<a href="https://colab.research.google.com/github/farhanadhilah/Applied_Deep_Learning/blob/main/AlexNet(BatchNormalization_dan_DenseLayer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**AlexNet**

> AlexNex adalah salah satu arsitektur jaringan syaraf dalam yang dikembangkan oleh Alex Krizhevsky, dkk dan memenagkan kompteisi ILSVRC pada tahun 2010.


AlexNet diklasifikasikan sebagai Convolutional Neural Network yang dapat menampung gambar berwarna dengan ukuran (224x224x3) dan dapat melatih lebih dari 62 juta parameter [1]. AlexNet menggunakan fungsi aktivasi ReLU.

##Tricks For Better Computation
Beberapa trik yang digunakan dalam arsitektur AlexNet adalah sebagai berikut [2]:

*   Menggunakan Batch Normalization (untuk mempercepat proses komputasi).
*   Umumnya digunakan non-overlapping pooling, namun pada AlexNet digunakan overlapping pooling. Jika non-overlapping pool digunakan, network hanya akan tetap fokus pada _dominant features_ dan nantinya akan mengarahkan pada masalah _overfitting_. Sementara itu jika menggunakan overlapping pool, informasi spasial akan tetap digunakan sehingga mencegah terjadinya _overfitting_.
*  Menggunakan _Data Augmentation_.
* Menggunakan teknik _Dropout_ (memperkecil _overfitting_).



---



1.   https://ichi.pro/id/memahami-alexnet-panduan-mendetail-36066595999914
2.   https://towardsdatascience.com/alexnet-8b05c5eb88d4


---

Code di bawah ini menggunakan arsitektur AlexNet dengan Batch Normalization dan Dense layer. Pada code di bawah ini digunakan model non-overlapping pool karena terdapat Batch Normalization yang memiliki efek regulasi dan sapat mencegah terjadinya _overfitting_. Source code: [2]


In [ ]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2 MB 4.1 kB/s 
     |████████████████████████████████| 2.9 MB 53.0 MB/s 
     |████████████████████████████████| 3.0 MB 41.5 MB/s 
     |████████████████████████████████| 454 kB 68.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalli

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

training_images = training_images[:1000]
training_labels = training_labels[:1000]
test_images = test_images[:100]
test_labels = test_labels[:100]

training_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), training_images).numpy()
test_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), test_images).numpy()

training_images = tf.image.resize(training_images, [224, 224]).numpy()
test_images = tf.image.resize(test_images, [224, 224]).numpy()

training_images = training_images.reshape(1000, 224, 224, 3)
training_images = training_images / 255.0
test_images = test_images.reshape(100, 224, 224, 3)
test_images = test_images / 255.0

training_labels = tf.keras.utils.to_categorical(training_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

num_len_train = int(0.8 * len(training_images))

ttraining_images = training_images[:num_len_train]
ttraining_labels = training_labels[:num_len_train]

valid_images = training_images[num_len_train:]
valid_labels = training_labels[num_len_train:]

training_images = ttraining_images
training_labels = ttraining_labels

In [ ]:
model = tf.keras.models.Sequential([
		tf.keras.layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', \
			kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01), \
			input_shape=(224, 224, 3)),

    tf.keras.layers.BatchNormalization(),
	
		tf.keras.layers.MaxPooling2D(3, strides=2),
		
		tf.keras.layers.Conv2D(256, (5, 5), activation='relu', \
			kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01), \
			bias_initializer='ones'),
		
		tf.keras.layers.MaxPooling2D(3, strides=2),
		
		tf.keras.layers.Conv2D(384, (3, 3), activation='relu', \
			kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01)),
		
		tf.keras.layers.Conv2D(384, (3, 3), activation='relu', \
			kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01), \
			bias_initializer='ones'),
		
		tf.keras.layers.Conv2D(384, (3, 3), activation='relu', \
			kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01), \
			bias_initializer='ones'),
		
		tf.keras.layers.MaxPooling2D(3, strides=2),
    		
		tf.keras.layers.Flatten(),
		
		tf.keras.layers.Dense(4096, kernel_initializer=\
		                      tf.random_normal_initializer(mean=0.0, stddev=0.01), \
				      bias_initializer='ones'),
		
		tf.keras.layers.Dropout(0.5),
		
		tf.keras.layers.Dense(4096, kernel_initializer=\
		                      tf.random_normal_initializer(mean=0.0, stddev=0.01), \
				      bias_initializer='ones'),
		
		tf.keras.layers.Dropout(0.5),
		
		tf.keras.layers.Dense(10, activation='softmax', \
				      kernel_initializer= \
				      tf.random_normal_initializer(mean=0.0, stddev=0.01))
	])


model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9), \
              loss='categorical_crossentropy', \
	      metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(5)])

print(model.summary())

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', \
                                            	factor=0.1, patience=1, \
						min_lr=0.00001)

model.fit(training_images, training_labels, batch_size=128, \
          validation_data=(valid_images, valid_labels), \
	  epochs=90, callbacks=[reduce_lr])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 54, 54, 96)        384       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 22, 22, 256)       614656    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 384)         885120    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 384)        